In [2]:
import re
import string
from collections import defaultdict, Counter

import nltk as nltk
import pandas as pd
from nltk import pos_tag, WordNetLemmatizer, SnowballStemmer
from nltk.corpus import wordnet

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\20184025\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\20184025\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\20184025\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\20184025\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\20184025\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
df_scientist_us_jobs = pd.read_csv('../data/data_scientist_united_states_job_postings_jobspikr.csv')
df_scientist_us_jobs.head()

,crawl_timestamp,url,job_title,category,company_name,city,state,country,inferred_city,inferred_state,...,job_description,job_type,salary_offered,job_board,geo,cursor,contact_email,contact_phone_number,uniq_id,html_job_description
0,2019-02-06 05:26:22 +0000,https://www.indeed.com/viewjob?jk=fd83355c2b23...,Enterprise Data Scientist I,Accounting/Finance,Farmers Insurance Group,Woodland Hills,CA,Usa,Woodland hills,California,...,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549432819114777,NaN,NaN,3b6c6acfcba6135a31c83bd7ea493b18,NaN
1,2019-02-06 05:33:41 +0000,https://www.dice.com/jobs/detail/Data-Scientis...,Data Scientist,NaN,Luxoft USA Inc,Middletown,NJ,Usa,Middletown,New jersey,...,We have an immediate opening for a Sharp Data ...,Undefined,NaN,dice,usa,1549432819122106,NaN,NaN,741727428839ae7ada852eebef29b0fe,NaN
2,2019-02-06 05:33:35 +0000,https://www.dice.com/jobs/detail/Data-Scientis...,Data Scientist,NaN,Cincinnati Bell Technology Solutions,New York,NY,Usa,New york,New york,...,Candidates should have the following backgroun...,Full Time,NaN,dice,usa,1549432819236156,NaN,NaN,cdc9ef9a1de327ccdc19cc0d07dbbb37,NaN
3,2019-02-06 05:33:42 +0000,https://www.indeed.com/viewjob?jk=841edd86ead2...,"Data Scientist, Aladdin Wealth Tech, Associate...",Accounting/Finance,BlackRock,New York,NY 10055 (Midtown area),Usa,New york,New york,...,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549432819259473,NaN,NaN,1c8541cd2c2c924f9391c7d3f526f64e,NaN
4,2019-02-06 05:48:23 +0000,https://job-openings.monster.com/senior-data-s...,Senior Data Scientist,biotech,CyberCoders,Charlotte,NC,Usa,Charlotte,North carolina,...,We are seeking an extraordinary Data Scientist...,Full Time,NaN,monster,usa,1549436429015957,NaN,NaN,445652a560a5441060857853cf267470,NaN


In [4]:
df_datascientist = pd.read_csv('../data/../data/DataScientist.csv', index_col=0)
df_datascientist.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1
1,1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1
2,2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True
3,3,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1
4,4,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\nDescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1


In [5]:
df_datascientist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3909 entries, 0 to 3908
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              3909 non-null   int64  
 1   Job Title          3909 non-null   object 
 2   Salary Estimate    3909 non-null   object 
 3   Job Description    3909 non-null   object 
 4   Rating             3909 non-null   float64
 5   Company Name       3909 non-null   object 
 6   Location           3909 non-null   object 
 7   Headquarters       3909 non-null   object 
 8   Size               3909 non-null   object 
 9   Founded            3909 non-null   int64  
 10  Type of ownership  3909 non-null   object 
 11  Industry           3909 non-null   object 
 12  Sector             3909 non-null   object 
 13  Revenue            3909 non-null   object 
 14  Competitors        3909 non-null   object 
 15  Easy Apply         3909 non-null   object 
dtypes: float64(1), int64(2),

In [6]:
df_datascientist['Job Description'][2]

"Decode_M\n\nhttps://www.decode-m.com/\n\nData Science Manager : Job Description\n\nWe’re hiring a Data Science Manager in our New York office to lead Decode_M’s Data Science team in developing new solutions for our clients and advancing the science of momentum through our proprietary product.\n\nThis position, based in New York City, requires an interest in managing Decode_M’s data science team, workflow and offerings. It demands excellent organizational skills, a natural curiosity, an eagerness to dive into the deep end, and a hunger to learn and grow while having a blast. Expect to learn a lot about cool companies, new products, and the latest in analytics, research and strategy.\n\nAbout You\nYou have an advanced degree in data science, mathematics, or another highly quantitative field, and are comfortable understanding and utilizing recent academic research in your work.\nYou have 3+ years of relevant work experience using applied statistics and/or machine learning. Bonus points i

In [7]:
def remove_urls(text):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    return re.sub(regex, '', text)

In [8]:
# remove urls
column_to_clean = 'Job Description'
df_datascientist[column_to_clean] = df_datascientist[column_to_clean].apply(remove_urls)
df_datascientist[column_to_clean]

0       ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...
1       At Noom, we use scientifically proven methods ...
2       Decode_M\n\n\n\nData Science Manager : Job Des...
3       Sapphire Digital seeks a dynamic and driven mi...
4       Director, Data Science - (200537)\nDescription...
                              ...                        
3904    About Us\n\nTachyon Technologies is a Digital ...
3905    Job description\nInterpret data, analyze resul...
3906    Job DescriptionThe Security Analytics Data Eng...
3907    The Security Analytics Data Engineer will inte...
3908    Help us transform patients' lives.\nAt UCB, we...
Name: Job Description, Length: 3909, dtype: object

In [9]:
df_datascientist[column_to_clean] = df_datascientist[column_to_clean].apply(lambda x: x.lower().strip())
df_datascientist[column_to_clean].head()

0    about hopper\n\nat hopper, we’re on a mission ...
1    at noom, we use scientifically proven methods ...
2    decode_m\n\n\n\ndata science manager : job des...
3    sapphire digital seeks a dynamic and driven mi...
4    director, data science - (200537)\ndescription...
Name: Job Description, dtype: object

In [10]:
stopwords = nltk.corpus.stopwords.words('english')
st = SnowballStemmer('english')
word_Lemmatized = WordNetLemmatizer()
tag_map = defaultdict(lambda: wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV


def lemmatize(words):
    tag_map = defaultdict(lambda: wordnet.NOUN)
    tag_map['J'] = wordnet.ADJ
    tag_map['V'] = wordnet.VERB
    tag_map['R'] = wordnet.ADV

    words = [word_Lemmatized.lemmatize(word, tag_map[tag[0]]) for word, tag in pos_tag(words)]

    return words


def tokenize_and_remove(text):
    tokens = nltk.tokenize.word_tokenize(text)
    # remove punctuation and stopwords
    tokens = list(
        filter(lambda token: token not in string.punctuation and token not in stopwords and token.isalpha(),
               tokens))
    # stemming
    # tokens = [st.stem(token) for token in tokens]

    # lemmatize the words
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    tokens = lemmatize(tokens)

    return tokens


tokenize_and_remove(df_datascientist['Job Description'][2])

['data',
 'science',
 'manager',
 'job',
 'description',
 'hire',
 'data',
 'science',
 'manager',
 'new',
 'york',
 'office',
 'lead',
 'data',
 'science',
 'team',
 'develop',
 'new',
 'solution',
 'client',
 'advance',
 'science',
 'momentum',
 'proprietary',
 'product',
 'position',
 'base',
 'new',
 'york',
 'city',
 'require',
 'interest',
 'manage',
 'data',
 'science',
 'team',
 'workflow',
 'offering',
 'demand',
 'excellent',
 'organizational',
 'skill',
 'natural',
 'curiosity',
 'eagerness',
 'dive',
 'deep',
 'end',
 'hunger',
 'learn',
 'grow',
 'blast',
 'expect',
 'learn',
 'lot',
 'cool',
 'company',
 'new',
 'product',
 'late',
 'analytics',
 'research',
 'strategy',
 'advance',
 'degree',
 'data',
 'science',
 'mathematics',
 'another',
 'highly',
 'quantitative',
 'field',
 'comfortable',
 'understanding',
 'utilizing',
 'recent',
 'academic',
 'research',
 'work',
 'year',
 'relevant',
 'work',
 'experience',
 'use',
 'applied',
 'statistic',
 'machine',
 'learn',


In [11]:
df_datascientist['new tokens'] = df_datascientist[column_to_clean].apply(tokenize_and_remove)
df_datascientist.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,new tokens
0,0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"about hopper\n\nat hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1,"[hopper, hopper, mission, make, book, travel, ..."
1,1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"at noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1,"[noom, use, scientifically, proven, method, he..."
2,2,Data Science Manager,$111K-$181K (Glassdoor est.),decode_m\n\n\n\ndata science manager : job des...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True,"[data, science, manager, job, description, hir..."
3,3,Data Analyst,$111K-$181K (Glassdoor est.),sapphire digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1,"[sapphire, digital, seek, dynamic, drive, data..."
4,4,"Director, Data Science",$111K-$181K (Glassdoor est.),"director, data science - (200537)\ndescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1,"[director, data, science, description, edelman..."


In [12]:
all_tokens = []
for _, value in df_datascientist['new tokens'].iteritems():
    all_tokens.extend(value)

word_count_job1: Counter = Counter(all_tokens)

In [13]:
word_count_job1.most_common(100)

[('data', 35645),
 ('experience', 21462),
 ('work', 14942),
 ('team', 10144),
 ('business', 10101),
 ('skill', 7072),
 ('year', 7013),
 ('use', 6676),
 ('include', 6473),
 ('science', 6182),
 ('development', 6003),
 ('model', 5851),
 ('analysis', 5785),
 ('develop', 5763),
 ('ability', 5628),
 ('require', 5231),
 ('technology', 5161),
 ('solution', 4973),
 ('product', 4971),
 ('design', 4965),
 ('support', 4956),
 ('system', 4860),
 ('project', 4840),
 ('knowledge', 4795),
 ('provide', 4711),
 ('process', 4670),
 ('job', 4666),
 ('analytics', 4646),
 ('learn', 4638),
 ('research', 4525),
 ('opportunity', 4322),
 ('service', 4263),
 ('strong', 4253),
 ('information', 4235),
 ('company', 4234),
 ('new', 4197),
 ('technical', 4040),
 ('management', 4030),
 ('requirement', 3997),
 ('scientist', 3835),
 ('build', 3825),
 ('tool', 3800),
 ('report', 3673),
 ('need', 3633),
 ('environment', 3560),
 ('application', 3463),
 ('engineering', 3431),
 ('customer', 3417),
 ('position', 3393),
 ('mac

In [14]:
word_count_job1['rational']

7

In [15]:
word_count_job1['modeling']

578

In [16]:
word_count_job1['python']

2636

In [16]:
import article_terms

# word count from lemmatized list
personality_traits_lemma = lemmatize(article_terms.all_personality_traits)
p_trait_lemma_count = {}
for p_trait in personality_traits_lemma:
    p_trait_lemma_count[p_trait] = word_count_job1[p_trait]

p_trait_lemma_count

# Direct text search of listed traits or skills

In [2]:
import os

path_output = lambda file_name: os.path.join('..', 'output', file_name)

In [18]:
# combine all job descriptions into 1 string
all_job_descriptions_1 = ""
for idx, value in df_datascientist['Job Description'].iteritems():
    all_job_descriptions_1 += value

all_job_descriptions_1 = all_job_descriptions_1.lower()
len(all_job_descriptions_1), all_job_descriptions_1[:100]

(13939723,
 'about hopper\n\nat hopper, we’re on a mission to make booking travel faster, easier, and more transpar')

In [19]:
all_job_descriptions_2 = ""
for idx, value in df_scientist_us_jobs['job_description'].iteritems():
    all_job_descriptions_2 += value

all_job_descriptions_2 = all_job_descriptions_2.lower()
len(all_job_descriptions_2), all_job_descriptions_2[:100]

(41434191,
 'read what people are saying about working here. \n\nwe are farmers!\n\njoin a team of diverse profession')

In [59]:
def clean_string_from_characters(text):
    return re.sub('(?![ -~]).', '', text)


all_job_descriptions_1 = clean_string_from_characters(all_job_descriptions_1)
all_job_descriptions_2 = clean_string_from_characters(all_job_descriptions_2)

In [60]:
# write to disk

def write_to_txt(text: str, file_name: str):
    with open(path_output(file_name), 'w') as fp:
        fp.write(text)


write_to_txt(all_job_descriptions_1, 'jop_descriptions_ds1.txt')
write_to_txt(all_job_descriptions_2, 'jop_descriptions_ds2_us.txt')

### Run analysis code from here

In [1]:
import os

path_output = lambda file_name: os.path.join('..', 'output', file_name)

def read_txt(file_name:str) -> str:
    with open(path_output(file_name), 'r') as fp:
        text = fp.read()
    return text

all_job_descriptions_1 = read_txt('jop_descriptions_ds1.txt')
all_job_descriptions_2 = read_txt('jop_descriptions_ds2_us.txt')

In [2]:
from typing import Optional
import article_terms
import pandas as pd

def count_traits_in_text(list_of_traits, output_csv: Optional[str] = None):
    traits_count = {}

    for trait in list_of_traits:
        traits_count[trait] = {'job1': all_job_descriptions_1.count(trait)}
        traits_count[trait]['job2'] = all_job_descriptions_2.count(trait)
        traits_count[trait]['total'] = traits_count[trait]['job1'] + \
                                         traits_count[trait]['job2']

    df_traits_count = pd.DataFrame(traits_count)
    df_traits_count = df_traits_count.T
    df_traits_count = df_traits_count.sort_values('total', ascending=False)

    if output_csv:
        df_traits_count.to_csv(path_output(output_csv))

    return df_traits_count

In [3]:
df_personality_traits_count = count_traits_in_text(article_terms.all_personality_traits, 'personality_traits.csv')
df_personality_traits_count

,job1,job2,total
technical,4519,12492,17011
analytical,2539,9067,11606
rational,787,2708,3495
mathematical,383,2474,2857
creative,615,2146,2761
logical,795,955,1750
curiosity,191,837,1028
creativity,246,768,1014
strong communication,188,623,811
critical thinking,158,509,667


In [4]:
df_seeked_traits = count_traits_in_text(article_terms.all_seeked_traits, 'seeked_traits.csv')
df_seeked_traits

,job1,job2,total
management,4143,10062,14205
communication,2443,6616,9059
presentation skills,185,670,855
mentoring,195,587,782
team player,237,471,708
risk management,149,530,679
critical thinking,158,509,667
time management,157,231,388
cooperation,27,56,83
software skills,26,17,43


In [5]:
df_soft_skills = count_traits_in_text(article_terms.all_soft_skills, 'soft_skills.csv')
df_soft_skills

,job1,job2,total
communication,2443,6616,9059
communication skills,1309,3501,4810
collaboration,655,1692,2347
problem-solving,446,1005,1451
curiosity,191,837,1028
team player,237,471,708
critical thinking,158,509,667
verbal and written communication skills,199,432,631
business acumen,142,407,549
storytelling,54,168,222


In [6]:
df_hard_skills = count_traits_in_text(article_terms.list_hard_skills_1, 'hard_skills.csv')
df_hard_skills

,job1,job2,total
machine learning,3195,20158,23353
statistics,1988,10707,12695
programming,1577,6683,8260
data visualization,713,3268,3981
deep learning,436,3414,3850
predictive modeling,268,2301,2569
coding,451,1413,1864
data wrangling,70,333,403
calculus,28,231,259
linear algebra,66,191,257


In [7]:
df_all_traits_skills = count_traits_in_text(article_terms.all_traits_skills, 'all_traits_skills.csv')
df_all_traits_skills

,job1,job2,total
machine learning,3195,20158,23353
technical,4519,12492,17011
management,4143,10062,14205
statistics,1988,10707,12695
analytical,2539,9067,11606
communication,2443,6616,9059
programming,1577,6683,8260
communication skills,1309,3501,4810
data visualization,713,3268,3981
deep learning,436,3414,3850
